In [ ]:
import pandas as pd

In [ ]:
ptio2_timings_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/pbto2_for_extraction.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'

In [ ]:
ptio2_timings_df = pd.read_csv(ptio2_timings_path)
drug_administration_df = pd.read_excel(drug_administration_path)

In [ ]:
ptio2_timings_df.head()

In [ ]:
drug_administration_df.head()

Number of patients / adminsitrations with concomitant monitoring

In [ ]:
format_str = "%Y %b %d %H:%M"

In [ ]:
from utils import format_date_column

ptio2_timings_df['Recording Start Time'] = format_date_column(ptio2_timings_df['Recording Start Time'])
ptio2_timings_df['Recording End Time'] = format_date_column(ptio2_timings_df['Recording End Time'])

In [ ]:
# ptio2_timings_df['first_pbtO2_recording_time'] if len < 6 then add :00
ptio2_timings_df['first_pbtO2_recording_time'] = ptio2_timings_df['first_pbtO2_recording_time'].apply(lambda x: x if len(x) > 5 else x + ':00')
ptio2_timings_df['last_pbtO2_recording_time'] = ptio2_timings_df['last_pbtO2_recording_time'].apply(lambda x: x if len(x) > 5 else x + ':00')

In [ ]:
# extract date from Recording Start Time, then add first_pbtO2_recording_day days and use time from first_pbtO2_recording_time
ptio2_timings_df['ptio2_start_datetime'] = pd.to_datetime(ptio2_timings_df['Recording Start Time'].str[:17], format=format_str).dt.date + pd.to_timedelta(ptio2_timings_df['first_pbtO2_recording_day'], unit='d') 
ptio2_timings_df['ptio2_start_datetime'] = ptio2_timings_df['ptio2_start_datetime'].astype(str) + ' ' + ptio2_timings_df['first_pbtO2_recording_time']

ptio2_timings_df['ptio2_end_datetime'] = pd.to_datetime(ptio2_timings_df['Recording Start Time'].str[:17], format=format_str).dt.date + pd.to_timedelta(ptio2_timings_df['last_pbtO2_recording_day'], unit='d')
ptio2_timings_df['ptio2_end_datetime'] = ptio2_timings_df['ptio2_end_datetime'].astype(str) + ' ' + ptio2_timings_df['last_pbtO2_recording_time']

In [ ]:
def is_administration_monitored(pat_nr, drug_start, drug_end):
    subj_df = ptio2_timings_df[ptio2_timings_df['pat_nr'] == pat_nr]
    subj_monitoring_start = pd.to_datetime(subj_df['ptio2_start_datetime'].values[0], format='%Y-%m-%d %H:%M:%S')
    subj_monitoring_end = pd.to_datetime(subj_df['ptio2_end_datetime'].values[0], format='%Y-%m-%d %H:%M:%S')
    # if drug start after monitoring start and drug end before monitoring end return True
    return subj_monitoring_start <= drug_end and subj_monitoring_end >= drug_start

In [ ]:
drug_administration_df['monitored'] = drug_administration_df.apply(lambda x: is_administration_monitored(x['pat_nr'], x['drug_start'], x['drug_end']), axis=1)

In [ ]:
monitored_drug_administration_df = drug_administration_df[drug_administration_df.monitored]

In [ ]:
n_patients = monitored_drug_administration_df.pat_nr.nunique()
n_drug_administrations = monitored_drug_administration_df.shape[0]

print(f'Number of patients {n_patients}')
print(f'Number of administrations {n_drug_administrations}')

In [ ]:
# drug_administration_df.to_excel('/Users/jk1/Downloads/drug_administrations_bis.xlsx', index=False)